In [23]:
import tensorflow as tf
from skimage import color, exposure, transform
import numpy, os
import glob, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [27]:




def preprocess_img(img):
    IMG_SIZE = 35
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE),mode='constant')

  

    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])

## root_dir :  Need to be a PATH with /(slash) not \ (backslash) example: C:/Users/Usuario/Documents/German-Traffic-Signs-Detector/images/FullIJCNN2013
def readImageAndProccess(root_dir):
    imgs = []
    labels = []

    all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
    ##print(all_img_paths)
    ##np.random.shuffle(all_img_paths)
    ##print(all_img_paths)
    for img_path in all_img_paths:
        ##print(img_path)
        img_path2=img_path.replace("\\","/")
        ##print(img_path2) io.imread                          
        img = preprocess_img(cv2.imread(img_path))
        label = get_class(img_path2)
        imgs.append(img)
        labels.append(label)
    ##print(labels) 

    
    
    ##print(X2.shape)
    ##print(nX2.shape)
    # Make one hot targets
    ##Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    ##print(len(labels))
    return imgs,labels

In [28]:
route="C:/Users/Usuario/Documents/German-Traffic-Signs-Detector/images/FullIJCNN2013"
imgs,labels=readImageAndProccess(route)
imgs_nparray = np.array(imgs, dtype='float32')

X = np.reshape(imgs_nparray, (imgs_nparray.shape[0], -1))
NUM_CLASSES = 43
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

#Onehot_encode Ylist
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_Ylist = onehot_encoder.fit_transform(integer_encoded)

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=0)

ntrain=len(x_train)
ntest=len(x_test)
dim= x_train.shape[1]
nclass=y_train.shape[1]
print(ntrain)
print(ntest)
print(dim)
print(nclass)


970
243
3675
43


In [29]:
#create a computation graph.
# Parameters of Logistic Regression
learning_rate   = 0.001
training_epochs = 1000
batch_size      = 10



In [30]:
#placeholder that will be fed at run time with a training minibatch
X_ = tf.placeholder(tf.float32, [None, dim])
Y_ = tf.placeholder(tf.float32, [None, nclass])
#Set model weights
W = tf.Variable(tf.zeros([dim, nclass]))
b = tf.Variable(tf.zeros([nclass]))

In [31]:
#Linear mapping
prediction = tf.matmul(X_, W) + b
model = tf.nn.softmax(tf.matmul(X_, W) + b)

#setup the minimisation
cost_function = -tf.reduce_sum(Y_*tf.log(prediction)) 
entropy = tf.nn.softmax_cross_entropy_with_logits( logits=prediction, labels=Y_)
loss = tf.reduce_mean(entropy) # computes the mean over examples in the batch
#
optimizer =tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [32]:
init = tf.global_variables_initializer()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        num_batch = int(ntrain/batch_size)
        # Loop over all batches
        for i in range(num_batch): 
            randidx = np.random.randint(ntrain, size=batch_size)
            batch_xs = x_train[randidx, :]
            batch_ys = y_train[randidx, :]
            # Fit training using batch data
            sess.run(optimizer, feed_dict={X_: batch_xs, Y_: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost_function, feed_dict={X_: batch_xs, Y_: batch_ys})/num_batch
    # test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print ("Accuracy:", accuracy.eval({X_: x_test, Y_: y_test}))
    sess.close()

    


Accuracy: 0.89711934
